In [59]:
#-------------------------------------------------------------------#
#1.Financial Model - LCOE Calculator Analysis
#-------------------------------------------------------------------#

In [ ]:
#Model Inputs
#Capital cost, $ (TCC)
#Fixed annual operating cost, $ (FOC)
#Variable operating cost, $/kWh (VOC)
#Fixed charge rate (FCR)
#Annual electricity production, kWh (AEP)

#Model Outputs
#LCOE, $/kWH

#References:
#https://sam.nrel.gov/financial-models/lcoe-calculator.html
#https://pysam-docs.readthedocs.io/en/latest/modules/Lcoefcr.html

In [78]:
#Dependencies
import PySAM.Lcoefcr as lc

In [79]:
#Instantiate the Lcoefcr model
model = lc.default("PVWattsLCOECalculator")

In [81]:
#Declare inputs variables
aep = 6500
tcc = 2000
fcr = 2
foc = 500
voc = 500

#Assemble in a dictionary
model_input = {"SimpleLCOE":{'annual_energy':aep,
    'capital_cost':tcc,
    'fixed_charge_rate':fcr,
    'fixed_operating_cost':foc,
    'variable_operating_cost':voc}}

#Assign the inputs to the model
mod.assign(model_input)

#Run the model
mod.execute()

In [84]:
#See results
mod.Outputs.lcoe_fcr

500.6923076923077

In [85]:
#Export model as dictionary
mod.export()

{'SimpleLCOE': {'annual_energy': 6500.0,
  'capital_cost': 2000.0,
  'fixed_charge_rate': 2.0,
  'fixed_operating_cost': 500.0,
  'variable_operating_cost': 500.0},
 'Outputs': {'lcoe_fcr': 500.6923076923077}}

In [60]:
#-------------------------------------------------------------------#
#2.Performance Model - PVWatts Module Analysis
#-------------------------------------------------------------------#

In [ ]:
#Model Inputs (Check documentation from the link in references)
#LocationAndResource => solar resource/weather file
#SystemDesign => not manditory but necessary, if not given as inputs there are default values
#AdjustmentFactors => not required

#References
#https://pysam-docs.readthedocs.io/en/latest/modules/Pvwattsv5.html

In [2]:
#Code to show the OUTPUTS of the model
import PySAM.Pvwattsv5 as pv
d = pv.default("PVWattsLCOECalculator")
d.LocationAndResource.solar_resource_file = "725_commonwealth_avenue_boston_ma_02215_42.350506_-71.107360_psmv3_60_2017.csv"
d.execute()
x = dict(d.Outputs.export())
#list(x.keys())

In [76]:
#Applying The Model

In [3]:
import PySAM.Pvwattsv5 as pv
d2 = pv.default("PVWattsLCOECalculator")
#d.LocationAndResource.solar_resource_file = "725_commonwealth_avenue_boston_ma_02215_42.350506_-71.107360_psmv3_60_2017.csv"

In [4]:
d2.SystemDesign.export()

{'array_type': 2.0,
 'azimuth': 180.0,
 'dc_ac_ratio': 1.2,
 'gcr': 0.4,
 'inv_eff': 96.0,
 'losses': 14.0757,
 'module_type': 0.0,
 'system_capacity': 20000.0,
 'tilt': 33.0}

In [5]:
model_inputs = {
    'LocationAndResource':
    {
        'solar_resource_file':"725_commonwealth_avenue_boston_ma_02215_42.350506_-71.107360_psmv3_60_2017.csv"
    },
    'SystemDesign':
      {
        'array_type': 2.0,
        'azimuth': 180.0,
        'dc_ac_ratio': 1.2,
        'gcr': 0.4,
        'inv_eff': 96.0,
        'losses': 14.0757,
        'module_type': 0.0,
        'system_capacity': 20000.0,
        'tilt': 33.0}
     }

In [6]:
d2.assign(model_inputs)

In [7]:
d2.execute()

In [13]:
#d2.Outputs.export()
#PR computation
outputs = d2.Outputs.export()
ac_annual = outputs['ac_annual']
system_inputs = d2.SystemDesign.export()
power_rating = system_inputs['system_capacity']
g_poa_annual = outputs['poa']
g_stc = 1000
A = [(x*power_rating)/g_stc for x in g_poa_annual]
pr = ac_annual/(sum(A))
pr

0.764053141101219

In [73]:
#-------------------------------------------------------------------#
#3.Performance Ratio Corrected Analysis
#-------------------------------------------------------------------#

In [3]:
'''
This part calculates the performance ratio of a location using expected values of energy generation, 
G_poa, T_cell from PVwatt. The weather file is from NSRDB. Assumptions for this analysis will be listed.
'''
#Reference => https://www.nrel.gov/docs/fy13osti/57991.pdf

'\nThis part calculates the performance ratio of a location using expected values of energy generation, \nG_poa, T_cell from PVwatt. The weather file is from NSRDB. Assumptions for this analysis will be listed.\n'

In [4]:
#1. Finding Measured AC electrical generation [kW] using PVWatt model (this will be power output measurement)
import PySAM.Pvwattsv5 as pv
model = pv.default("PVWattsLCOECalculator")

model_inputs = {
    'LocationAndResource':
    {
        'solar_resource_file':"33.45_-111.98_33.450547_-111.980443_psmv3_60_2017.csv"
    },
    'SystemDesign':
      {
        'array_type': 2.0,
        'azimuth': 180.0,
        'dc_ac_ratio': 1.2,
        'gcr': 0.4,
        'inv_eff': 96.0,
        'losses': 14.0757,
        'module_type': 0.0,
        'system_capacity': 20000.0,
        'tilt': 33.0}
     }

model.assign(model_inputs)
model.execute()

outputs = model.Outputs.export()
ac_annual = outputs['ac_annual']

In [5]:
#2. Summation of installed module's power rating [kW]/nameplate rating
system_inputs = model.SystemDesign.export()
power_rating = system_inputs['system_capacity']
power_rating

20000.0

In [6]:
#3. Measured Plane of Array Irradiance [kW/m2] (which will be pyranometer data)
g_poa_annual = outputs['poa']

In [7]:
#4. Irradiance at STC
g_stc = 1000

In [8]:
#5. Temperature correction
#Temperature coefficient for power %/C
delta = 0.5 
#Cell temperature per hour from SAM
tcell = outputs['tcell']
#Average cell temperature
y = 0 
for x in range(8760):
    y = y + g_poa_annual[x]*tcell[x]
tcell_avg = y/sum(g_poa_annual)

A = [tcell_avg - x for x in tcell]
B = [(delta/100)*x for x in A]
corr = [1 - x for x in B]

In [9]:
#6. Compute the Performance Ratio
A = [(x*power_rating)/g_stc for x in g_poa_annual]
pr = ac_annual/(sum(A))
pr

0.7227043372981035

In [10]:
#7. Compute the Performance Ratio Corrected
A = [(x*power_rating)/g_stc for x in g_poa_annual]
B = 0
for x in range(8760):
    B = B + A[x]*corr[x]

pr_corr = ac_annual/B
pr_corr

0.7227043372981007

In [90]:
import pandas as pd

In [106]:
x = "725_commonwealth_avenue_boston_ma_02215_42.350506_-71.107360_psmv3_60_2017.csv"
y = "albuquerque_weather_file.csv"
df = pd.read_csv(x)
dict(df)['Longitude']

0       -71.1
1         DHI
2           0
3           0
4           0
5           0
6           0
7           0
8           0
9          14
10         50
11         65
12         72
13         75
14         74
15         68
16         56
17         35
18          0
19          0
20          0
21          0
22          0
23          0
24          0
25          0
26          0
27          0
28          0
29          0
        ...  
8732        0
8733        0
8734        0
8735        0
8736        0
8737        0
8738        0
8739        0
8740        0
8741        0
8742        0
8743        0
8744        0
8745       21
8746       49
8747       67
8748       80
8749       84
8750      146
8751      131
8752       90
8753       44
8754        0
8755        0
8756        0
8757        0
8758        0
8759        0
8760        0
8761        0
Name: Longitude, Length: 8762, dtype: object

In [99]:
for keys in dict(df):
    print(keys)

Unnamed: 0
Year
Month
Day
Hour
Minute
GHI
DHI
DNI
Wind Speed
Temperature
Solar Zenith Angle
